In [128]:
import pandas as pd
import statsmodels.api as sm
import datetime as datetime
import numpy as np

def zscore(df, col):
    col_mean = df[col].rolling(window=10).mean()
    col_std = df[col].rolling(window=10).std()
    return (df[col] - col_mean)/col_std

df = pd.read_csv('./data/11-8/to_trump_2M_dates_res/daily_plutchik_onehot.csv', parse_dates=['month/day'])
approval_rating_df = pd.read_csv('./data/approval_polllist.csv', parse_dates=["enddate"])

df["Anticipation Z-Score"] = zscore(df, "Anticipation")
df["Trust Z-Score"] = zscore(df, "Trust")
df["Surprise Z-Score"] = zscore(df, "Surprise")
df["Sadness Z-Score"] = zscore(df, "Sadness")
df["Joy Z-Score"] = zscore(df, "Joy")
df["Fear Z-Score"] = zscore(df, "Fear")
df["Disgust Z-Score"] = zscore(df, "Disgust")
df["Anger Z-Score"] = zscore(df, "Anger")

start_date = datetime.datetime(2017, 5, 15, 0, 0)
end_date = datetime.datetime(2017, 11, 17, 0, 0)

truncated_df = df[(df["month/day"] > start_date) & (df["month/day"] < end_date)]
pd.date_range(start = '2017-05-15', end = '2017-11-17' ).difference(truncated_df["month/day"])
# print(truncated_df.index)

DatetimeIndex(['2017-05-15', '2017-06-22', '2017-11-17'], dtype='datetime64[ns]', freq=None)

In [ ]:
unnormalized_df = truncated_df[['Anticipation', 'Trust', 'Sadness', 'Joy', 'Fear', 'Disgust', 'Anger']]
unnormalized_extra_df = truncated_df[['Anticipation', 'Trust', 'Surprise', 'Sadness', 'Joy', 'Fear', 'Disgust', 'Anger']]
truncated_df = truncated_df[['Anticipation Z-Score', 'Trust Z-Score', 'Surprise Z-Score', 
                             'Sadness Z-Score', 'Joy Z-Score', 'Disgust Z-Score', 'Anger Z-Score']]
truncated_without_surprise_df = truncated_df[['Anticipation Z-Score', 'Trust Z-Score', 
                             'Sadness Z-Score', 'Joy Z-Score', 'Disgust Z-Score', 'Anger Z-Score']]
truncated_df.head()

In [117]:
truncated_approvals = approval_rating_df.loc[approval_rating_df['pollster'] == 'Gallup']
truncated_approvals = truncated_approvals.loc[truncated_approvals['subgroup'] == 'All polls']
truncated_approvals = truncated_approvals[(truncated_approvals['enddate'] > start_date) & (truncated_approvals['enddate'] <= end_date)]
truncated_approvals = truncated_approvals['adjusted_approve']
truncated_approvals.head()

439    39.07889
445    39.07889
447    39.07889
452    38.07889
456    39.07889
Name: adjusted_approve, dtype: float64

In [118]:
truncated_approvals = truncated_approvals.reset_index(drop=True)
truncated_df = truncated_df.reset_index(drop=True)
truncated_without_surprise_df = truncated_without_surprise_df.reset_index(drop=True)
unnormalized_df = unnormalized_df.reset_index(drop=True)
unnormalized_extra_df = unnormalized_extra_df.reset_index(drop=True)

In [119]:
model = sm.OLS(truncated_approvals, truncated_df).fit()

In [120]:
predictions = model.predict(truncated_df)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       adjusted_approve   R-squared (uncentered):                   0.011
Model:                            OLS   Adj. R-squared (uncentered):             -0.028
Method:                 Least Squares   F-statistic:                             0.2818
Date:                Wed, 20 Nov 2019   Prob (F-statistic):                       0.960
Time:                        14:28:27   Log-Likelihood:                         -931.75
No. Observations:                 184   AIC:                                      1878.
Df Residuals:                     177   BIC:                                      1900.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Anticipation Z-Score     0.2864      3.021      0.095      0.925      -5.676       6.249
Trust Z-Score            3.3091      4.854      0.682      0.496      -6.269      12.888
Surprise Z-Score         3.5039      4.769      0.735      0.463      -5.908      12.916
Sadness Z-Score          1.6014      3.526      0.454      0.650      -5.358       8.561
Joy Z-Score              2.7971      3.509      0.797      0.426      -4.127       9.721
Disgust Z-Score          1.8549      3.985      0.465      0.642      -6.010       9.720
Anger Z-Score            4.3958      3.479      1.263      0.208      -2.470      11.262
==============================================================================
Omnibus:                        7.083   Durbin-Watson:                   0.017
Prob(Omnibus):                  0.029   Jarque-Bera (JB):                9.074
Skew:                          -0.255   Prob(JB):                       0.0107
Kurtosis:                       3.961   Cond. No.                         3.99
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [121]:
model = sm.OLS(truncated_approvals, unnormalized_df).fit()

In [122]:
# predictions = model.predict(unnormalized_df)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       adjusted_approve   R-squared (uncentered):                   0.997
Model:                            OLS   Adj. R-squared (uncentered):              0.997
Method:                 Least Squares   F-statistic:                              8549.
Date:                Wed, 20 Nov 2019   Prob (F-statistic):                   2.69e-220
Time:                        14:28:27   Log-Likelihood:                         -396.75
No. Observations:                 184   AIC:                                      807.5
Df Residuals:                     177   BIC:                                      830.0
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Anticipation    16.1514     34.390      0.470      0.639     -51.717      84.019
Trust           42.7391      2.080     20.552      0.000      38.635      46.843
Sadness         62.8037     16.887      3.719      0.000      29.477      96.130
Joy             72.0979      7.464      9.660      0.000      57.369      86.827
Fear            47.7235      7.596      6.283      0.000      32.733      62.714
Disgust         30.8956     16.689      1.851      0.066      -2.040      63.831
Anger          -17.9919     16.953     -1.061      0.290     -51.447      15.464
==============================================================================
Omnibus:                        9.688   Durbin-Watson:                   0.777
Prob(Omnibus):                  0.008   Jarque-Bera (JB):                9.944
Skew:                           0.481   Prob(JB):                      0.00693
Kurtosis:                       3.609   Cond. No.                         125.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [123]:
model = sm.OLS(truncated_approvals, unnormalized_extra_df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       adjusted_approve   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     2.573
Date:                Wed, 20 Nov 2019   Prob (F-statistic):             0.0150
Time:                        14:28:28   Log-Likelihood:                -339.83
No. Observations:                 184   AIC:                             695.7
Df Residuals:                     176   BIC:                             721.4
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Anticipation   -13.2639     25.424     -0.522      0.603     -63.439      36.911
Trust           37.2038      1.596     23.318      0.000      34.055      40.353
Surprise        49.4073      4.024     12.279      0.000      41.466      57.349
Sadness         51.8676     12.461      4.162      0.000      27.276      76.459
Joy             38.0352      6.154      6.181      0.000      25.890      50.180
Fear            26.7141      5.846      4.569      0.000      15.176      38.252
Disgust         48.2185     12.364      3.900      0.000      23.818      72.619
Anger           24.0654     12.939      1.860      0.065      -1.469      49.600
==============================================================================
Omnibus:                        4.012   Durbin-Watson:                   0.570
Prob(Omnibus):                  0.134   Jarque-Bera (JB):                3.585
Skew:                           0.288   Prob(JB):                        0.167
Kurtosis:                       3.370   Cond. No.                         129.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [124]:
model = sm.OLS(truncated_approvals, truncated_without_surprise_df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       adjusted_approve   R-squared (uncentered):                   0.008
Model:                            OLS   Adj. R-squared (uncentered):             -0.025
Method:                 Least Squares   F-statistic:                             0.2395
Date:                Wed, 20 Nov 2019   Prob (F-statistic):                       0.963
Time:                        14:28:28   Log-Likelihood:                         -932.03
No. Observations:                 184   AIC:                                      1876.
Df Residuals:                     178   BIC:                                      1895.
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Anticipation Z-Score     0.3401      3.016      0.113      0.910      -5.612       6.293
Trust Z-Score            0.7960      3.439      0.231      0.817      -5.990       7.583
Sadness Z-Score          0.6207      3.260      0.190      0.849      -5.812       7.053
Joy Z-Score              2.4318      3.469      0.701      0.484      -4.413       9.277
Disgust Z-Score          0.3873      3.444      0.112      0.911      -6.409       7.184
Anger Z-Score            3.3246      3.155      1.054      0.293      -2.901       9.550
==============================================================================
Omnibus:                        0.674   Durbin-Watson:                   0.013
Prob(Omnibus):                  0.714   Jarque-Bera (JB):                0.652
Skew:                          -0.143   Prob(JB):                        0.722
Kurtosis:                       2.942   Cond. No.                         2.21
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""